# Strong Acid Cation (H-form) Ion Exchange Report

**Generated:** 2025-09-18T14:50:44.097215
**Run ID:** 20250918_143042_171fe159
**Resin Type:** SAC


In [ ]:
# Parameters injected by papermill
# This cell will be tagged for hiding in HTML export

## Setup Environment

In [ ]:
# Setup Environment - Common imports and configuration
import json
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import handcalcs for equation rendering
try:
    from handcalcs import render
except ImportError:
    print("Installing handcalcs...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'handcalcs'])
    from handcalcs import render

# Import forallpeople for units
try:
    import forallpeople as si
except ImportError:
    print("Installing forallpeople...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'forallpeople'])
    import forallpeople as si

# Setup SI units environment
si.environment('default', top_level=True)

# Import plotting libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['font.size'] = 11

try:
    import plotly.graph_objects as go
    import plotly.express as px
    USE_PLOTLY = True
except ImportError:
    USE_PLOTLY = False
    print("Plotly not available, using matplotlib")

print("Environment setup complete")
print(f"Project root: {project_root}")
print(f"Run ID: {run_id}")

In [ ]:
# Load simulation data
sim_data = simulation
design = design_inputs
resin_info = resin_metadata

# Extract key sections
performance = sim_data.get('performance', {})
ion_tracking = sim_data.get('ion_tracking', {})
mass_balance = sim_data.get('mass_balance', {})
economics = sim_data.get('economics', {})
breakthrough_data = sim_data.get('breakthrough_data', {})

print(f"Loaded simulation data for {resin_info['display_name']}")
print(f"Breakthrough at {performance.get('service_bv_to_target', 0):.1f} BV")

## Basis Of Design

### Feed Water Composition

The following table presents the feed water analysis used for the ion exchange system design:

In [ ]:
# Display water composition table
feed_ions = ion_tracking.get('feed_mg_l', {})

# Create water quality dataframe
water_quality = []
for ion, conc in feed_ions.items():
    water_quality.append({
        'Ion': ion,
        'Concentration (mg/L)': f"{conc:.2f}",
        'Concentration (meq/L)': f"{conc/20:.3f}"  # Simplified - should use actual equiv weights
    })

df_water = pd.DataFrame(water_quality)
display(df_water.style.set_caption('Feed Water Composition'))

# Calculate total hardness
ca_mg_l = feed_ions.get('Ca_2+', 0)
mg_mg_l = feed_ions.get('Mg_2+', 0)
total_hardness = ca_mg_l * 2.5 + mg_mg_l * 4.1  # as CaCO3

print(f"\nTotal Hardness: {total_hardness:.1f} mg/L as CaCO3")
print(f"Target Effluent Hardness: {performance.get('effluent_hardness_mg_l_caco3', 5):.1f} mg/L as CaCO3")

### Design Targets

The ion exchange system is designed to meet the following performance targets:

In [ ]:
# Display design targets
targets = [
    ['Effluent Hardness', f"{performance.get('effluent_hardness_mg_l_caco3', 5):.1f} mg/L as CaCO3"],
    ['Service Flow Rate', f"{design.get('flow_m3_hr', 100):.1f} m³/hr"],
    ['Minimum Service Run', f"{performance.get('service_hours', 24):.1f} hours"],
    ['Resin Type', resin_info['display_name']]
]

# Only add alkalinity if it exists and is not None
alk_value = performance.get('effluent_alkalinity_mg_l_caco3')
if alk_value is not None:
    targets.append(['Effluent Alkalinity', f"{alk_value:.1f} mg/L as CaCO3"])

df_targets = pd.DataFrame(targets, columns=['Parameter', 'Value'])
display(df_targets.style.set_caption('Design Performance Targets'))

## Sac Vessel Sizing

### Hydraulic Design Calculations

The following calculations determine the required vessel dimensions based on hydraulic constraints:

In [ ]:
# Service Flow Rate Design (SAC)
# Using standard heuristics for RO pretreatment

from math import pi

# Design parameters (from simulation inputs)
Q_m3_hr = design.get('flow_m3_hr', 100)      # Design flow rate in m³/hr
SV_hr = 16                                    # Service velocity (bed volumes/hour)
LV_max_m_hr = 25                             # Maximum linear velocity in m/hr

# Calculate required bed volume
V_bed_m3 = Q_m3_hr / SV_hr                   # Bed volume required in m³

# Calculate minimum cross-sectional area
A_min_m2 = Q_m3_hr / LV_max_m_hr             # Minimum area for linear velocity in m²

# Convert to liters for display
V_bed_L = V_bed_m3 * 1000                    # Volume in liters

print(f"Design flow rate Q = {Q_m3_hr:.1f} m³/hr")
print(f"Required bed volume = {V_bed_m3:.2f} m³ = {V_bed_L:.0f} L")
print(f"Minimum cross-sectional area = {A_min_m2:.2f} m²")

In [ ]:
# Vessel Diameter Selection
# Standard vessel diameters for containerized systems

# Minimum diameter from area
D_min_m = (4 * A_min_m2 / pi) ** 0.5

# Round up to standard diameter (0.1m increments)
D_selected_m = 2.3                           # Selected standard diameter in meters

# Actual cross-sectional area
A_actual_m2 = pi * (D_selected_m / 2) ** 2

# Actual linear velocity
LV_actual_m_hr = Q_m3_hr / A_actual_m2

print(f"Minimum diameter = {D_min_m:.2f} m")
print(f"Selected standard diameter = {D_selected_m:.2f} m")
print(f"Actual linear velocity = {LV_actual_m_hr:.1f} m/hr")

In [ ]:
# Bed Depth Calculation

# Calculate bed depth from volume
h_bed_m = V_bed_m3 / A_actual_m2

# Minimum bed depth constraint
h_bed_min_m = 0.75                           # Minimum for proper distribution

# Select actual bed depth (round to 0.05m)
h_bed_final_m = 1.5                          # Selected bed depth in meters

# Recalculate actual bed volume
V_bed_actual_m3 = A_actual_m2 * h_bed_final_m

# Service flow rate in BV/hr
SV_actual_hr = Q_m3_hr / V_bed_actual_m3

print(f"Calculated bed depth = {h_bed_m:.2f} m")
print(f"Selected bed depth = {h_bed_final_m:.2f} m")
print(f"Actual bed volume = {V_bed_actual_m3:.2f} m³")
print(f"Actual service velocity = {SV_actual_hr:.1f} BV/hr")

In [ ]:
# Freeboard and Total Height

# Freeboard for backwash expansion (100% for conservative design)
expansion = 1.0                               # 100% expansion factor
h_freeboard_m = h_bed_final_m * expansion

# Total vessel height
h_vessel_m = h_bed_final_m + h_freeboard_m + 0.1  # Plus distributor/collector space

# Aspect ratio check
aspect_ratio = h_bed_final_m / D_selected_m

print(f"Freeboard height = {h_freeboard_m:.2f} m")
print(f"Total vessel height = {h_vessel_m:.2f} m")
print(f"Aspect ratio (L/D) = {aspect_ratio:.2f}")

In [ ]:
# Summary of vessel design
vessel_summary = [
    ['Vessel Diameter', f"{D_selected_m:.2f}", 'm'],
    ['Bed Depth', f"{h_bed_final_m:.2f}", 'm'],
    ['Bed Volume', f"{V_bed_actual_m3:.2f}", 'm³'],
    ['Freeboard Height', f"{h_freeboard_m:.2f}", 'm'],
    ['Total Vessel Height', f"{h_vessel_m:.2f}", 'm'],
    ['Linear Velocity', f"{LV_actual_m_hr:.1f}", 'm/hr'],
    ['Service Velocity', f"{SV_actual_hr:.1f}", 'BV/hr'],
    ['Aspect Ratio (L/D)', f"{aspect_ratio:.2f}", '-']
]

df_vessel = pd.DataFrame(vessel_summary, columns=['Parameter', 'Value', 'Units'])
display(df_vessel.style.set_caption('SAC Vessel Design Summary'))

# Design verification
print("\n✓ Design Verification:")
if LV_actual_m_hr <= 25:
    print(f"  Linear velocity {LV_actual_m_hr:.1f} ≤ 25 m/hr ✓")
else:
    print(f"  ⚠️ Linear velocity {LV_actual_m_hr:.1f} > 25 m/hr")

if h_bed_final_m >= 0.75:
    print(f"  Bed depth {h_bed_final_m:.2f} ≥ 0.75 m ✓")
else:
    print(f"  ⚠️ Bed depth {h_bed_final_m:.2f} < 0.75 m")

if D_selected_m <= 2.4:
    print(f"  Diameter {D_selected_m:.2f} ≤ 2.4 m (fits in container) ✓")
else:
    print(f"  ⚠️ Diameter {D_selected_m:.2f} > 2.4 m (exceeds container)")

## Breakthrough Analysis

### Breakthrough Curve Analysis

The following plot shows the predicted breakthrough behavior from PHREEQC simulation:

In [ ]:
# Load breakthrough data if available
if breakthrough_data and 'bed_volumes' in breakthrough_data:
    bv = breakthrough_data['bed_volumes']
    hardness = breakthrough_data.get('hardness_mg_l', [])
    ca = breakthrough_data.get('ca_mg_l', [])
    mg = breakthrough_data.get('mg_mg_l', [])
    na = breakthrough_data.get('na_mg_l', [])
    
    # Find service endpoint
    target_hardness = breakthrough_data.get('target_hardness', 3.0)
    breakthrough_bv = breakthrough_data.get('breakthrough_bv', performance.get('service_bv_to_target', 0))
    
    print(f"Breakthrough at {breakthrough_bv:.1f} BV for {target_hardness:.1f} mg/L target")
elif breakthrough_curve_path:
    # Try to load from CSV file
    try:
        df_breakthrough = pd.read_csv(breakthrough_curve_path)
        bv = df_breakthrough['bed_volumes'].values
        hardness = df_breakthrough['hardness_mg_l'].values
        print(f"Loaded breakthrough data from {breakthrough_curve_path}")
    except:
        print("No breakthrough data available")
        bv = hardness = None
else:
    print("No breakthrough data available for plotting")
    bv = hardness = None

In [ ]:
# Create breakthrough curve plot
if bv is not None and len(bv) > 0:
    if USE_PLOTLY:
        # Interactive Plotly plot
        fig = go.Figure()
        
        # Add hardness trace
        fig.add_trace(go.Scatter(
            x=bv[:len(hardness)],
            y=hardness,
            mode='lines',
            name='Total Hardness',
            line=dict(color='blue', width=2)
        ))
        
        # Add target line
        fig.add_hline(
            y=target_hardness,
            line_dash="dash",
            line_color="red",
            annotation_text=f"Target: {target_hardness} mg/L"
        )
        
        # Add breakthrough point
        fig.add_vline(
            x=breakthrough_bv,
            line_dash="dash",
            line_color="green",
            annotation_text=f"Breakthrough: {breakthrough_bv:.1f} BV"
        )
        
        fig.update_layout(
            title='Ion Exchange Breakthrough Curve',
            xaxis_title='Bed Volumes',
            yaxis_title='Hardness (mg/L as CaCO₃)',
            height=500,
            hovermode='x unified'
        )
        
        fig.show()
    else:
        # Static matplotlib plot
        plt.figure(figsize=(10, 6))
        plt.plot(bv[:len(hardness)], hardness, 'b-', linewidth=2, label='Total Hardness')
        plt.axhline(y=target_hardness, color='r', linestyle='--', label=f'Target: {target_hardness} mg/L')
        plt.axvline(x=breakthrough_bv, color='g', linestyle='--', label=f'Breakthrough: {breakthrough_bv:.1f} BV')
        
        plt.xlabel('Bed Volumes')
        plt.ylabel('Hardness (mg/L as CaCO₃)')
        plt.title('Ion Exchange Breakthrough Curve')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.xlim(0, min(300, max(bv) if bv else 300))
        plt.ylim(0, max(100, max(hardness)*1.1 if hardness else 100))
        plt.show()
else:
    print("⚠️ No breakthrough curve data available for visualization")

In [ ]:
# Performance metrics summary
# Get bed volume from simulation details or vessel parameters
vessel_params = simulation.get('input', {}).get('vessel', {})
bed_volume_m3 = vessel_params.get('bed_volume_m3')
if not bed_volume_m3:
    # Calculate from dimensions if available
    diameter = vessel_params.get('diameter_m', 2.0)
    bed_depth = vessel_params.get('bed_depth_m', 2.5)
    from math import pi
    bed_volume_m3 = pi * (diameter/2)**2 * bed_depth

service_bv = performance.get('service_bv_to_target', 0)
treated_volume_m3 = service_bv * bed_volume_m3

performance_summary = [
    ['Service to Breakthrough', f"{service_bv:.1f}", 'BV'],
    ['Service Run Time', f"{performance.get('service_hours', 0):.1f}", 'hours'],
    ['Treated Water Volume', f"{treated_volume_m3:.0f}", 'm³'],
    ['Capacity Utilization', f"{performance.get('capacity_utilization_percent', 0):.1f}", '%'],
    ['Effluent Quality', f"{performance.get('effluent_hardness_mg_l_caco3', 0):.1f}", 'mg/L as CaCO₃']
]

df_performance = pd.DataFrame(performance_summary, columns=['Metric', 'Value', 'Units'])
display(df_performance.style.set_caption('Service Cycle Performance'))

## Mass Balance

### Mass Balance and Regeneration

Material balance for the service and regeneration cycles:

In [ ]:
# Mass balance data
mass_bal = mass_balance

# Create mass balance summary
mass_summary = [
    ['Hardness Removed', f"{mass_bal.get('hardness_removed_kg_caco3', 0):.2f}", 'kg CaCO₃/cycle'],
    ['Regenerant Consumed', f"{mass_bal.get('regenerant_kg_cycle', 0):.1f}", 'kg NaCl/cycle'],
    ['Backwash Volume', f"{mass_bal.get('backwash_m3_cycle', 0):.1f}", 'm³/cycle'],
    ['Rinse Volume', f"{mass_bal.get('rinse_m3_cycle', 0):.1f}", 'm³/cycle'],
    ['Total Waste Volume', f"{mass_bal.get('waste_m3_cycle', 0):.1f}", 'm³/cycle'],
    ['Mass Balance Closure', f"{mass_bal.get('closure_percent', 99):.1f}", '%']
]

df_mass = pd.DataFrame(mass_summary, columns=['Parameter', 'Value', 'Units'])
display(df_mass.style.set_caption('Regeneration Mass Balance'))

In [ ]:
# Regeneration efficiency calculations

# Parameters from mass balance
hardness_removed_kg = mass_bal.get('hardness_removed_kg_caco3', 41.6)
regenerant_used_kg = mass_bal.get('regenerant_kg_cycle', 416)

# Stoichiometric requirement (2 mol NaCl per mol CaCO3)
MW_CaCO3 = 100.09  # g/mol
MW_NaCl = 58.44    # g/mol

# Theoretical regenerant needed (kg)
regen_stoich_kg = hardness_removed_kg * (2 * MW_NaCl / MW_CaCO3)

# Regeneration efficiency
if regenerant_used_kg > 0:
    regen_efficiency = (regen_stoich_kg / regenerant_used_kg) * 100  # percent
else:
    regen_efficiency = 0

print(f"Theoretical regenerant required: {regen_stoich_kg:.1f} kg")
print(f"Actual regenerant used: {regenerant_used_kg:.1f} kg")
print(f"Regeneration efficiency: {regen_efficiency:.1f}%")
print(f"Excess regenerant factor: {regenerant_used_kg/regen_stoich_kg if regen_stoich_kg > 0 else 0:.1f}x")

In [ ]:
# Ion removal summary
if ion_tracking and 'removal_percent' in ion_tracking:
    removals = ion_tracking['removal_percent']
    
    removal_data = []
    for ion, removal in removals.items():
        if removal > 0:  # Only show ions that were removed
            removal_data.append({
                'Ion': ion,
                'Feed (mg/L)': f"{ion_tracking['feed_mg_l'].get(ion, 0):.1f}",
                'Effluent (mg/L)': f"{ion_tracking['effluent_mg_l'].get(ion, 0):.2f}",
                'Removal (%)': f"{removal:.1f}"
            })
    
    if removal_data:
        df_removal = pd.DataFrame(removal_data)
        display(df_removal.style.set_caption('Ion Removal Performance'))
    else:
        print("No ion removal data available")
else:
    print("Ion tracking data not available")

## Economics Summary

### Economic Analysis

Capital and operating cost breakdown for the ion exchange system:

In [ ]:
# Check if economics data is available
if economics and 'capital_cost_usd' in economics:
    has_economics = True
else:
    has_economics = False
    print("⚠️ Economic analysis not available in simulation results")

In [ ]:
if has_economics:
    # Capital cost breakdown
    unit_costs = economics.get('unit_costs', {})
    
    capex_breakdown = [
        ['Vessels', f"${unit_costs.get('vessels_usd', 0):,.0f}"],
        ['Initial Resin', f"${unit_costs.get('resin_initial_usd', 0):,.0f}"],
        ['Pumps & Valves', f"${unit_costs.get('pumps_usd', 0):,.0f}"],
        ['Instrumentation', f"${unit_costs.get('instrumentation_usd', 0):,.0f}"],
    ]
    
    if unit_costs.get('degasser_usd'):
        capex_breakdown.append(['CO₂ Degasser', f"${unit_costs.get('degasser_usd', 0):,.0f}"])
    
    # Add installation factor
    subtotal = sum(v for k, v in unit_costs.items() if k != 'installation_factor' and v)
    install_factor = unit_costs.get('installation_factor', 2.5)
    capex_breakdown.append(['Installation (×{:.1f})'.format(install_factor), 
                           f"${(subtotal * (install_factor - 1)):,.0f}"])
    capex_breakdown.append(['**TOTAL CAPEX**', f"**${economics.get('capital_cost_usd', 0):,.0f}**"])
    
    df_capex = pd.DataFrame(capex_breakdown, columns=['Item', 'Cost (USD)'])
    display(df_capex.style.set_caption('Capital Cost Breakdown'))

In [ ]:
if has_economics:
    # Operating cost breakdown
    opex_breakdown = [
        ['Regenerant (NaCl)', f"${economics.get('regenerant_cost_usd_year', 0):,.0f}/yr"],
        ['Resin Replacement', f"${economics.get('resin_replacement_cost_usd_year', 0):,.0f}/yr"],
        ['Energy', f"${economics.get('energy_cost_usd_year', 0):,.0f}/yr"],
    ]
    
    if economics.get('waste_disposal_cost_usd_year'):
        opex_breakdown.append(['Waste Disposal', 
                              f"${economics.get('waste_disposal_cost_usd_year', 0):,.0f}/yr"])
    
    opex_breakdown.append(['**TOTAL OPEX**', 
                          f"**${economics.get('operating_cost_usd_year', 0):,.0f}/yr**"])
    
    df_opex = pd.DataFrame(opex_breakdown, columns=['Item', 'Cost'])
    display(df_opex.style.set_caption('Operating Cost Breakdown'))

In [ ]:
if has_economics:
    # Key economic metrics
    lcow = economics.get('lcow_usd_m3', 0)
    sec = economics.get('sec_kwh_m3', 0)
    
    print("\n📊 Key Economic Metrics:")
    print(f"  • Levelized Cost of Water (LCOW): ${lcow:.3f}/m³")
    print(f"  • Specific Energy Consumption: {sec:.3f} kWh/m³")
    print(f"  • Payback Period: {economics.get('capital_cost_usd', 0) / economics.get('operating_cost_usd_year', 1):.1f} years (simplified)")
    
    # Cost per 1000 gallons for US reference
    lcow_per_kgal = lcow * 3.785  # 1000 gallons = 3.785 m³
    print(f"  • Cost per 1000 gallons: ${lcow_per_kgal:.2f}")

## Conclusions

### Conclusions and Recommendations

Based on the simulation and analysis:

In [ ]:
# Generate conclusions based on performance
conclusions = []

# Performance assessment
service_bv = performance.get('service_bv_to_target', 0)
if service_bv > 200:
    conclusions.append("✓ Excellent service capacity achieved (>200 BV)")
elif service_bv > 150:
    conclusions.append("✓ Good service capacity achieved (150-200 BV)")
elif service_bv > 100:
    conclusions.append("⚠️ Moderate service capacity (100-150 BV) - consider optimization")
else:
    conclusions.append("❌ Low service capacity (<100 BV) - review design")

# Effluent quality
eff_hardness = performance.get('effluent_hardness_mg_l_caco3', 0)
if eff_hardness <= 5:
    conclusions.append(f"✓ Effluent hardness ({eff_hardness:.1f} mg/L) meets RO pretreatment requirements")
else:
    conclusions.append(f"⚠️ Effluent hardness ({eff_hardness:.1f} mg/L) may be too high for RO")

# Capacity utilization
capacity_util = performance.get('capacity_utilization_percent', 0)
if capacity_util > 80:
    conclusions.append(f"✓ High resin capacity utilization ({capacity_util:.1f}%)")
elif capacity_util > 60:
    conclusions.append(f"✓ Good resin capacity utilization ({capacity_util:.1f}%)")
else:
    conclusions.append(f"⚠️ Low resin capacity utilization ({capacity_util:.1f}%) - potential for optimization")

# Economics (if available)
if has_economics:
    lcow = economics.get('lcow_usd_m3', 0)
    if lcow < 0.15:
        conclusions.append(f"✓ Competitive water cost (${lcow:.3f}/m³)")
    elif lcow < 0.25:
        conclusions.append(f"✓ Reasonable water cost (${lcow:.3f}/m³)")
    else:
        conclusions.append(f"⚠️ High water cost (${lcow:.3f}/m³) - consider optimization")

# Display conclusions
print("\n".join(conclusions))

### Recommendations for Optimization

In [ ]:
# Generate recommendations
recommendations = []

if service_bv < 150:
    recommendations.append("• Consider increasing regenerant dose to improve capacity")
    recommendations.append("• Evaluate counter-current regeneration for better efficiency")

if capacity_util < 70:
    recommendations.append("• Optimize regeneration conditions (concentration, flow rate)")
    recommendations.append("• Consider resin upgrade to higher capacity grade")

# WAC-specific recommendations
if 'WAC' in resin_info.get('code', ''):
    if 'effluent_alkalinity_mg_l_caco3' in performance:
        eff_alk = performance['effluent_alkalinity_mg_l_caco3']
        if eff_alk > 10:
            recommendations.append("• Consider CO₂ stripping for alkalinity reduction")
    recommendations.append("• Monitor pH carefully during regeneration")

# Energy optimization
if has_economics:
    sec = economics.get('sec_kwh_m3', 0)
    if sec > 0.1:
        recommendations.append("• Evaluate pump efficiency and pressure drop reduction")

if recommendations:
    print("Optimization Opportunities:")
    print("\n".join(recommendations))
else:
    print("✓ System is well-optimized with current parameters")

---
*Report generated using IX Design MCP Server with PHREEQC chemistry engine*